# Zero offset correction: calibrate pressure sensor

## Load and inspect data
Load pickle file and inspect contents

In [ ]:
import os
import pickle
import pandas as pd

# Import necessary pyologger utilities
from pyologger.utils.config_manager import ConfigManager
from pyologger.utils.event_manager import *
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata
from pyologger.plot_data.plotter import *
from pyologger.process_data.sampling import upsample
from pyologger.calibrate_data.tag2animal import *
from pyologger.calibrate_data.zoc import *
from pyologger.plot_data.plotter import plot_depth_correction

# Change the current working directory to the root directory
# os.chdir("/Users/fbar/Documents/GitHub/pyologger")
os.chdir("/Users/jessiekb/Documents/GitHub/pyologger")

root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")
color_mapping_path = os.path.join(root_dir, "color_mappings.json")

# Verify the current working directory
print(f"Current working directory: {root_dir}")

In [ ]:
# Initialize the info class
metadata = Metadata()
metadata.fetch_databases(verbose=False)

# Save databases
deployment_db = metadata.get_metadata("deployment_DB")
logger_db = metadata.get_metadata("logger_DB")
recording_db = metadata.get_metadata("recording_DB")
animal_db = metadata.get_metadata("animal_DB")

# Assuming you have the metadata and deployment_db loaded:
datareader = DataReader()
deployment_folder, deployment_id = datareader.check_deployment_folder(deployment_db, data_dir)
config_manager = ConfigManager(deployment_folder=deployment_folder, deployment_id=deployment_id)

In [ ]:
current_processing_step = "Processing Step 01 IN PROGRESS."
config_manager.add_to_config("current_processing_step", current_processing_step)

In [ ]:
# Load the data_reader object from the pickle file
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

with open(pkl_path, 'rb') as file:
    data_pkl = pickle.load(file)

for logger_id, info in data_pkl.logger_info.items():
    sampling_frequency = info.get('datetime_metadata', {}).get('fs', None)
    if sampling_frequency is not None:
        # Format the sampling frequency to 5 significant digits
        print(f"Sampling frequency for {logger_id}: {sampling_frequency} Hz")
    else:
        print(f"No sampling frequency available for {logger_id}")

## Find dives
Involves a zero offset correction with `zoc`

In [ ]:
?smooth_downsample_derivative

In [ ]:
# Load the depth and temperature data
depth_data = data_pkl.sensor_data['pressure']['pressure']
depth_datetime = data_pkl.sensor_data['pressure']['datetime']
depth_fs = data_pkl.sensor_info['pressure']['sampling_frequency']
temp_data = data_pkl.sensor_data['temperature']['temp']
temp_fs = data_pkl.sensor_info['temperature']['sampling_frequency']

In [ ]:
# Define and load configuration settings for dive detection
dive_detection_params = {
    "first_deriv_threshold": 0.1,          # Threshold for the first derivative of depth (meters/second)
    "min_duration": 30,                   # Minimum duration for a surface interval (seconds)
    "depth_threshold": 5,                 # Maximum depth to qualify as a surface interval (meters)
    "apply_temp_correction": False,       # Apply temperature correction during zero offset correction (True/False)
    "min_depth_threshold": 0.5,           # Minimum depth to start/end a dive (meters)
    "dive_duration_threshold": 10,        # Minimum duration to qualify as a dive (seconds)
    "smoothing_window": 5,                # Smoothing window size for dive detection (seconds)
    "downsampled_sampling_rate": 1        # Target sampling rate after downsampling (Hz)
}

# Add dive detection settings to the configuration file
config_manager.add_to_config(entries=dive_detection_params, section="dive_detection_settings")

# Retrieve dive detection parameters from the configuration
dive_detection_settings = config_manager.get_from_config(
    variable_names=list(dive_detection_params.keys()), 
    section="dive_detection_settings"
)

# Print the loaded parameters
print("Loaded Dive Detection Parameters:")
for key, value in dive_detection_settings.items():
    print(f"{key}: {value}")

# Process depth data: downsample and calculate the first derivative
depth_processing_params = {
    "original_sampling_rate": depth_fs,
    "downsampled_sampling_rate": dive_detection_settings["downsampled_sampling_rate"]
}
first_derivative, downsampled_depth = smooth_downsample_derivative(depth_data, **depth_processing_params)
depth_downsampled_datetime = depth_datetime[::int(depth_fs / dive_detection_settings["downsampled_sampling_rate"])]

# Detect flat chunks (potential surface intervals)
flat_chunk_params = {
    "depth": downsampled_depth,
    "datetime_data": depth_downsampled_datetime,
    "first_derivative": first_derivative,
    "threshold": dive_detection_settings["first_deriv_threshold"],
    "min_duration": dive_detection_settings["min_duration"],
    "depth_threshold": dive_detection_settings["depth_threshold"],
    "original_sampling_rate": depth_fs,
    "downsampled_sampling_rate": dive_detection_settings["downsampled_sampling_rate"]
}
flat_chunks = detect_flat_chunks(**flat_chunk_params)

# Apply zero offset correction
zoc_params = {
    "depth": downsampled_depth,
    "temp": temp_data.values if temp_data is not None else None,
    "flat_chunks": flat_chunks
}
corrected_depth_temp, corrected_depth_no_temp, depth_correction = apply_zero_offset_correction(**zoc_params)

# Choose corrected depth based on temperature correction setting
if dive_detection_settings["apply_temp_correction"]:
    corrected_depth = corrected_depth_temp
else:
    corrected_depth = corrected_depth_no_temp

# Detect dives in the corrected depth data
dive_detection_params = {
    "depth_series": corrected_depth,
    "datetime_data": depth_downsampled_datetime,
    "min_depth_threshold": dive_detection_settings["min_depth_threshold"],
    "sampling_rate": dive_detection_settings["downsampled_sampling_rate"],
    "duration_threshold": dive_detection_settings["dive_duration_threshold"],
    "smoothing_window": dive_detection_settings["smoothing_window"]
}
dives = find_dives(**dive_detection_params)
# Calculate dive duration in seconds
dives['dive_duration'] = (dives['end_time'] - dives['start_time']).dt.total_seconds()

# Log transformations
transformation_log = [
    f"downsampled_{dive_detection_settings['downsampled_sampling_rate']}Hz",
    f"smoothed_{dive_detection_settings['smoothing_window']}s",
    f"ZOC_settings__first_deriv_threshold_{dive_detection_settings['first_deriv_threshold']}mps__min_duration_{dive_detection_settings['min_duration']}s__depth_threshold_{dive_detection_settings['depth_threshold']}m",
    f"DIVE_detection_settings__min_depth_threshold_{dive_detection_settings['min_depth_threshold']}m__dive_duration_threshold_{dive_detection_settings['dive_duration_threshold']}s__smoothing_window_{dive_detection_settings['smoothing_window']}"
]

# Outputs
print(f"Number of potential surface intervals detected: {len(flat_chunks)}")
print(f"Number of dives detected: {len(dives)}")
print("Transformation Log:", transformation_log)


### Save data

In [ ]:
dives

In [ ]:
# Generate and update dive events
data_pkl.event_data = create_state_event(
    state_df=dives,
    key='dive',
    value_column='max_depth',
    start_time_column='start_time',
    duration_column='dive_duration', # in seconds
    description='dive_start',
    existing_events=data_pkl.event_data  # Pass existing events for overwrite and concatenation
)

# Update event_info with unique keys
data_pkl.event_info = list(data_pkl.event_data['key'].unique())

In [ ]:
data_pkl.event_info

In [ ]:
data_pkl.event_data

## Save data to pickle

In [ ]:
# Create the derived_from_sensors list
derived_from_sensors = ["pressure"]
original_name = 'Temp-corrected Depth (m)' if dive_detection_settings["apply_temp_correction"] else 'Corrected Depth (m)'

# Save the corrected depth back to the data structure
depth_df = pd.DataFrame({
    'datetime': depth_downsampled_datetime,
    'depth': corrected_depth
})
derived_info = {
    "channels": ["depth"],
    "metadata": {
            'depth': {'original_name': original_name,
                    'unit': 'm',
                    'sensor': 'pressure'}
    },
    "derived_from_sensors": derived_from_sensors.append("temperature") if dive_detection_settings["apply_temp_correction"] else derived_from_sensors,
    "transformation_log": transformation_log.append("temperature_correction") if dive_detection_settings["apply_temp_correction"] else derived_from_sensors
}

data_pkl.derived_data['depth'] = depth_df
data_pkl.derived_info['depth'] = derived_info

In [ ]:
# Load time range for plotting from the configuration file
time_settings = config_manager.get_from_config(
    variable_names=["earliest_common_start_time", "latest_common_end_time"],
    section="settings"
)

# Convert the loaded time settings into datetime objects
plot_start_time = pd.Timestamp(time_settings["earliest_common_start_time"]).to_pydatetime()
plot_end_time = pd.Timestamp(time_settings["latest_common_end_time"]).to_pydatetime()

# Ensure the loaded times are valid
if not (plot_start_time and plot_end_time):
    raise ValueError("Invalid time range loaded from configuration file.")

# Define the time range for the plot
time_range = (plot_start_time, plot_end_time)

# Define the sensors and derived data to plot
sensors_to_plot = ['pressure']
derived_data_to_plot = ['depth']

# Define annotations or notes to include in the plot (optional, e.g., detected dives or flat intervals)
note_annotations = {
    'dive': {'signal': 'depth', 'symbol': 'triangle-down', 'color': 'blue'}  # Example for dive events
}

# Use plot_tag_data_interactive5 to create the interactive plot
fig = plot_tag_data_interactive5(
    data_pkl=data_pkl,
    sensors=sensors_to_plot,
    derived_data_signals=derived_data_to_plot,
    time_range=time_range,
    note_annotations=note_annotations,
    color_mapping_path=color_mapping_path,  # Path to the color mapping JSON file
    target_sampling_rate=1  # Adjust this if you want the data plotted at a specific resolution
)

# Display the plot
fig.show()  # If running in a Jupyter notebook


In [ ]:
data_pkl.save_to_netcdf(os.path.join(deployment_folder, 'outputs/deployment_data_processed.nc'))

In [ ]:
?DataReader.save_to_netcdf

In [ ]:
import xarray as xr

# Define the path to the NetCDF file
netcdf_path = os.path.join(deployment_folder, 'outputs', 'deployment_data_processed.nc')

# Open the NetCDF file
data = xr.open_dataset(netcdf_path)

# Display the contents of the NetCDF file
display(data)

In [ ]:
# Dynamically generate and print the statement
print(f"`{', '.join(derived_from_sensors)}` sensor data was transformed into derived data `depth` by applying these transformations: {', '.join(transformation_log)}")
current_processing_step = "Processing Step 01. Calibration of pressure sensor complete and dives analyzed."
print(current_processing_step)

In [ ]:
# Add or update the current_processing_step for the specified deployment
print(current_processing_step)
config_manager.add_to_config("current_processing_step", current_processing_step)

# Optional: save new pickle file
with open(pkl_path, 'wb') as file:
        pickle.dump(data_pkl, file)
print("Pickle file updated.")